In [2]:
import pandas as pd
import re

In [3]:
df = pd.read_csv("./data_cleaned.csv")
df.head()

,location,price,year,type,garage,balcony,roof,garden,avg_price_year_type
0,xghajra,256000,2016,HOUSE,True,True,True,False,184208.088235
1,swieqi,365000,2016,HOUSE,True,False,False,False,184208.088235
2,qormi,300000,2016,HOUSE,True,False,True,True,184208.088235
3,msida,186300,2016,HOUSE,False,True,True,False,184208.088235
4,san gwann,210000,2016,HOUSE,False,True,True,False,184208.088235


In [4]:
df.drop(['avg_price_year_type'], inplace=True, axis=1)

In [5]:
df.head()

,location,price,year,type,garage,balcony,roof,garden
0,xghajra,256000,2016,HOUSE,True,True,True,False
1,swieqi,365000,2016,HOUSE,True,False,False,False
2,qormi,300000,2016,HOUSE,True,False,True,True
3,msida,186300,2016,HOUSE,False,True,True,False
4,san gwann,210000,2016,HOUSE,False,True,True,False


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [7]:
df.sort_values('year', inplace=True)

In [8]:
categorical_features = ['location', 'type']
one_hot = OneHotEncoder(drop='first')
one_hot_df = one_hot.fit_transform(df[categorical_features]).toarray()
one_hot_df = pd.DataFrame(one_hot_df, columns=one_hot.get_feature_names_out(categorical_features))
one_hot_df.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
one_hot_df.head()


,location_bahar ic-caghaq,location_balzan,location_birgu,location_birguma,location_birkirkara,location_birzebbugia,location_blata l-bajda,location_bugibba,location_burmarrad,location_cospicua,...,location_xghajra,location_xlendi,location_zabbar,location_zebbug,location_zejtun,location_zurrieq,type_BUNGALOW,type_HOUSE,type_MAISONETTE,type_VILLA
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
df_encoded = pd.concat([df.drop(categorical_features, axis=1), one_hot_df], axis=1)
df_encoded.shape

(1759, 88)

In [10]:
for column in ['garage', 'balcony', 'roof', 'garden', 'year', 'price']:
    df_encoded[column] = df[column].astype(int)

In [11]:
X = df_encoded.drop('price', axis=1)
y = df_encoded['price']

In [12]:
train_size = int(len(df_encoded) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [12]:
model = SARIMAX(y_train, exog=X_train, order=(1, 0, 0), seasonal_order=(0, 0, 0, 0))
model_fit = model.fit(disp=False)
y_pred = model_fit.predict(start=len(X_train), end=len(X_train) + len(X_test) - 1, exog=X_test)
import numpy as np
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: ", rmse)

RMSE:  322931.147507526


In [13]:
model = SARIMAX(y_train, exog=X_train, order=(1, 1, 0), seasonal_order=(0, 0, 0, 0))
model_fit = model.fit(disp=False)
y_pred = model_fit.predict(start=len(X_train), end=len(X_train) + len(X_test) - 1, exog=X_test)
import numpy as np
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: ", rmse)

RMSE:  236729.26411446818


In [14]:
model = SARIMAX(y_train, exog=X_train, order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
model_fit = model.fit(disp=False)
y_pred = model_fit.predict(start=len(X_train), end=len(X_train) + len(X_test) - 1, exog=X_test)
import numpy as np
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: ", rmse)

RMSE:  322925.71419000416


In [19]:
import itertools
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))

In [20]:
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [16]:
import numpy as np
best_aic = np.inf
best_pdq = None
best_seasonal_pdq = None
temp_model = None
from tqdm import tqdm

In [21]:
for param in tqdm(pdq):
    for param_seasonal in tqdm(pdqs):
        
        try:
            temp_model = SARIMAX(y_train,
                                 exog = X_train,
                                 order = param,
                                 seasonal_order = param_seasonal,
                                 enforce_stationarity=False,
                                 enforce_invertibility=False)
            results = temp_model.fit()

            # print("SARIMAX{}x{}12 - AIC:{}".format(param, param_seasonal, results.aic))
            if results.aic < best_aic:
                best_aic = results.aic
                best_pdq = param
                best_seasonal_pdq = param_seasonal
        except:
            #print("Unexpected error:", sys.exc_info()[0])
            continue
print("Best SARIMAX{}x{}12 model - AIC:{}".format(best_pdq, best_seasonal_pdq, best_aic))

# Fit the model with the best params
best_model = SARIMAX(y_train, exog=X_train, order=best_pdq, seasonal_order=best_seasonal_pdq)
best_model_fit = best_model.fit(disp=False)

# Make prediction
y_pred = best_model_fit.predict(start=len(X_train), end=len(X_train) + len(X_test) - 1, exog=X_test)

# Calculate RMSE
mse = mean_squared_error(y_test, y_pred)

  0%|          | 0/27 [00:00<?, ?it/s]

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           88     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.53576D+01    |proj g|=  5.29883D-07

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   88      0      1      0     0     0   5.299D-07   1.536D+01
  F =   15.357638552872443     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            


 This problem is unconstrained.
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           89     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.21315D+01    |proj g|=  3.25863D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   89      2      5      1     0     0   1.505D-04   1.213D+01
  F =   12.131465900871754     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


...

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           90     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.22139D+01    |proj g|=  1.76585D-03


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   90      3      6      1     0     0   1.451D-05   1.221D+01
  F =   12.213854860525629     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           91     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.21099D+01    |proj g|=  7.75398D-03
